# Introduction
## Learn and estimate the Home Credit Default Risk competition on Kaggle.


In [70]:
#predefines and import
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Problem 1
## Confirmation of competition contents

1. What to learn and what to predict?

    - We're using the data set to learn and predict whether our customer given their attributes have the ability to payoff our loan in time.

2. What kind of file to create and submit to Kaggle?

    - We summit a file that show our predicted probability of customer being type 1 (slow on dept payment) for all test indexes.
    - Ex of an index: Id = 1230459, Probability = 0.7

3. What kind of index value will be used to evaluate the submissions?
    - Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

### ROC and AUC [Reference](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc#:~:text=AUC%3A%20Area%20Under%20the%20ROC,to%20(1%2C1).)

-  **ROC**: curve (receiver operating characteristic curve) is a curve graph in 2d plane of TPR and FPR. The curve is plot of our classifier's performance under different decision thresholds

- **AUC**: Area under curve (integral of our ROC curve). This is used to draw the graph as well as for evaluating our classification model. A model with AUC = 0 means it does not have any kind of correct prediction and vice versa for AUC = 1. 

# Problem 2
## Learning and verification

## Load

In [71]:
init_houseprice_data = pd.read_csv('../Week 3/creditinfo_train.csv')
init_houseprice_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
print(f'Data shape: {init_houseprice_data.shape}')

Data shape: (307511, 122)


## Null check

In [73]:
row_count = init_houseprice_data.shape[0]
null_count = init_houseprice_data.isnull().sum()
null_cols = init_houseprice_data.columns[null_count > 0]
print('All columns with null values: ', len(null_cols),' columns')
null_count.sort_values(ascending=False).head(15)

All columns with null values:  67  columns


COMMONAREA_MEDI             214865
COMMONAREA_AVG              214865
COMMONAREA_MODE             214865
NONLIVINGAPARTMENTS_MODE    213514
NONLIVINGAPARTMENTS_AVG     213514
NONLIVINGAPARTMENTS_MEDI    213514
FONDKAPREMONT_MODE          210295
LIVINGAPARTMENTS_MODE       210199
LIVINGAPARTMENTS_AVG        210199
LIVINGAPARTMENTS_MEDI       210199
FLOORSMIN_AVG               208642
FLOORSMIN_MODE              208642
FLOORSMIN_MEDI              208642
YEARS_BUILD_MEDI            204488
YEARS_BUILD_MODE            204488
dtype: int64

## Finding unusable columns
Naively, I think that if a column is > 50% null, it's certainly unusable

In [74]:
null_count_normalized = init_houseprice_data[null_cols].isnull().sum() / row_count * 100
unusable_cols = null_count_normalized.index[null_count_normalized > 50]
print('Unusable Cols: ', len(unusable_cols))
unusable_cols

Unusable Cols:  41


Index(['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
       'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
       'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG',
       'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE',
       'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
       'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
       'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
       'WALLSMATERIAL_MODE'],
      dtype='object')

### After First Purge

In [75]:
first_purge = init_houseprice_data.drop(columns= unusable_cols)
print('After first Purge, columns left: ', first_purge.shape[1])
first_purge.columns

After first Purge, columns left:  81


Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
       'ORGANIZATION_TYPE', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
       'YEARS_BEGINEXPLUATATION_AVG', 'FLOOR

## Checking Attribute types and fill up null
I'm just going to go for the naive aproach, filling the remaining missing cells with the **MODE** of each corresponding column. 

MODE is important here cause MEAN value may not be in the attribute's domain.

In [89]:
mode_fill_values = dict()
for col in first_purge:
    mode_fill_values[col] = first_purge[col].mode()[0]
prob2_data = first_purge.fillna(mode_fill_values)
display(prob2_data.head())
print('Data Remains Null Value? ', prob2_data.isna().sum().any())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Data Remains Null Value?  False


## Building and Testing Model

### Label Encoding

In [93]:
categorical_columns = prob2_data.columns[prob2_data.dtypes == 'object']
prob2_data[categorical_columns].nunique()

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
EMERGENCYSTATE_MODE            2
dtype: int64

In [94]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()

for col in categorical_columns:
    prob2_data[col] = label_enc.fit_transform(prob2_data[col])
prob2_data[categorical_columns]

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,EMERGENCYSTATE_MODE
0,0,1,0,1,6,7,4,3,1,8,6,5,0
1,0,0,0,0,1,4,1,1,1,3,1,39,0
2,1,1,1,1,6,7,4,3,1,8,1,11,0
3,0,0,0,1,6,7,4,0,1,8,6,5,0
4,0,1,0,1,6,7,4,3,1,3,4,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,1,0,0,6,7,4,2,5,14,4,43,0
307507,0,0,0,1,6,3,4,5,1,8,1,57,0
307508,0,0,0,1,6,7,1,2,1,10,4,39,0
307509,0,0,0,1,6,1,4,1,1,8,6,3,0


### Speparating, Standardize

In [98]:
X = prob2_data.drop('TARGET',axis = 1)
Y = prob2_data['TARGET']

from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

print(f'Sizes: xtrain,ytrain: {x_train.shape, y_train.shape}, xtest,ytest: {x_test.shape,y_test.shape}')

Sizes: xtrain,ytrain: ((246008, 80), (246008,)), xtest,ytest: ((61503, 80), (61503,))


### Using SVC 

In [99]:
from sklearn.svm import SVC
cls = SVC()
cls.fit(x_train,y_train)
pred = cls.predict(x_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print('Accuracy: ', acc)

## Investigating other lesser null columns

In [84]:
second_threshold = 20
null_count_normalized = first_purge.isnull().sum() / row_count * 100
lesser_null_cols = null_count_normalized.index[null_count_normalized > second_threshold]
print(f'Colums with more than {second_threshold}% or ~{second_threshold*row_count/100} null tuple (of total {row_count}): {len(lesser_null_cols)}')
lesser_null_cols

Colums with more than 20% or ~61502.2 null tuple (of total 307511): 9


Index(['OCCUPATION_TYPE', 'YEARS_BEGINEXPLUATATION_AVG', 'FLOORSMAX_AVG',
       'YEARS_BEGINEXPLUATATION_MODE', 'FLOORSMAX_MODE',
       'YEARS_BEGINEXPLUATATION_MEDI', 'FLOORSMAX_MEDI', 'TOTALAREA_MODE',
       'EMERGENCYSTATE_MODE'],
      dtype='object')

### Description on above attributes:
"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized


## Confession
### To be completely honest, i think my top down approach is not working, due to reasons below
- The data set is wayy to big with so many attributes

- I dont understand many of the attributes presented, examples:
    - The group of attributes describing normalized values of where the customer live (APARTMENTS_AVG	BASEMENTAREA_AVG, YEARS_BEGINEXPLUATATION_AVG, ... ) - This have 3 sufixes (AVG, MODE, MEDI)
    - Flag document attributes
    - And many others i have not read through
### So ima opting for another aproach for this assignment
- Choosing relevant attributes that i understand and 'straight forward' in nature
- Clean them up and then make a model.
    